In [ ]:
class IndependentDQNTrainer:
    """Handles training logic for the DQN."""
    def __init__(self, state_size, action_size, learning_rate=0.001, gamma=0.99, memory_size=10000, batch_size=64):
        self.gamma = gamma
        self.batch_size = batch_size

        # Replay buffer
        self.memory = deque(maxlen=memory_size)

        # Q-Network
        self.model = DQNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.criterion = nn.MSELoss()

    def store_experience(self, state, action, reward, next_state, done):
        """Store experience in the replay buffer."""
        self.memory.append((state, action, reward, next_state, done))

    def train(self):
        """Train the DQN using experience replay."""
        if len(self.memory) < self.batch_size:
            return
        batch = random.sample(self.memory, self.batch_size)

        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(states).to(device)
        actions = torch.LongTensor(actions).to(device)
        rewards = torch.FloatTensor(rewards).to(device)
        next_states = torch.FloatTensor(next_states).to(device)
        dones = torch.FloatTensor(dones).to(device)

        # Current Q-values
        q_values = self.model(states).gather(1, actions.unsqueeze(1)).squeeze()

        # Target Q-values
        with torch.no_grad():
            next_q_values = self.model(next_states).max(1)[0]
            target_q_values = rewards + (1 - dones) * self.gamma * next_q_values

        # Loss and optimization
        loss = self.criterion(q_values, target_q_values)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

# IndependentDQNTrainer

---

## 주요 기능
1. **경험 리플레이 버퍼**: 
   - 경험 데이터를 저장하고 무작위 샘플링을 통해 학습 데이터를 제공합니다.
2. **Q-네트워크 학습**:
   - 저장된 경험을 기반으로 현재 Q-값과 타겟 Q-값의 차이를 줄이기 위해 네트워크를 최적화합니다.

---

## 클래스 메서드

### `__init__(self, state_size, action_size, learning_rate=0.001, gamma=0.99, memory_size=10000, batch_size=64)`
- **설명**: 학습에 필요한 주요 구성 요소를 초기화합니다.
- **파라미터**:
  - `state_size` (int): 상태 공간의 크기.
  - `action_size` (int): 행동 공간의 크기.
  - `learning_rate` (float): Q-네트워크의 학습 속도. (기본값: `0.001`)
  - `gamma` (float): 할인율, 미래 보상의 중요도를 결정. (기본값: `0.99`)
  - `memory_size` (int): 경험 리플레이 버퍼의 최대 크기. (기본값: `10000`)
  - `batch_size` (int): 학습에 사용할 샘플 크기. (기본값: `64`)
- **구성 요소**:
  - **경험 리플레이 버퍼** (`self.memory`): 최신 경험을 저장하며 최대 크기를 초과하면 오래된 데이터를 제거.
  - **Q-네트워크** (`self.model`): 상태를 입력받아 Q-값을 출력하는 PyTorch 기반 신경망.
  - **손실 함수** (`self.criterion`): 현재 Q-값과 타겟 Q-값 간의 차이를 계산 (MSE 사용).
  - **최적화 알고리즘** (`self.optimizer`): 학습을 위한 Adam 옵티마이저.

---

### `store_experience(self, state, action, reward, next_state, done)`
- **설명**: 주어진 경험 데이터를 경험 리플레이 버퍼에 저장합니다.
- **파라미터**:
  - `state`: 현재 상태.
  - `action`: 에이전트가 취한 행동.
  - `reward`: 행동 결과로 얻은 보상.
  - `next_state`: 다음 상태.
  - `done`: 에피소드 종료 여부 (`True` 또는 `False`).

---

### `train(self)`
- **설명**: 경험 리플레이 버퍼에서 샘플 데이터를 가져와 Q-네트워크를 학습시킵니다.
- **로직**:
  1. 경험 리플레이 버퍼 크기가 `batch_size`보다 작으면 학습을 건너뜁니다.
  2. `batch_size`만큼 랜덤으로 샘플링하여 학습 데이터를 준비합니다.
  3. 현재 Q-값과 타겟 Q-값을 계산:
     - **현재 Q-값**: 모델이 예측한 Q-값.
     - **타겟 Q-값**: 보상과 다음 상태의 최대 Q-값의 조합 (`rewards + (1 - dones) * gamma * next_q_values`).
  4. MSE 손실을 계산하고 역전파를 통해 Q-네트워크를 업데이트.